In [2]:
import pandas as pd
import numpy as np
import os
import random

In [7]:
# data split
data = pd.read_csv("data/data-2022-6-1.csv")
msk = np.random.rand(len(data)) <= 0.80
train = data[msk]
test = data[~msk]
print(len(train), len(test))
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

96 23


In [3]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import pickle

In [4]:
def GenerateFP(smiles,names):
	FP_array = np.zeros((len(smiles),1024))
	BitInfoDic = {}
	fps = []
	for i,smi in enumerate(smiles):
		m1 = Chem.MolFromSmiles(smi)
		fp1 = AllChem.GetMorganFingerprintAsBitVect(m1,2,nBits=1024)
		fps.append(fp1)
		DataStructs.ConvertToNumpyArray(fp1,FP_array[i,:])
		# Explain the Fingerprints
		info={} 
		AllChem.GetMorganFingerprint(m1,2,bitInfo=info)	# with radius=2, are roughly equivalent to ECFP4 and FCFP4
		# print info
		BitInfoDic[smi]=info
	return FP_array,BitInfoDic

In [7]:
datatype = 'test'
f = pd.read_csv('data/%s.csv'%datatype)
smiles = f['SMILES']
names = f['Name']
f.set_index('Name')
FP_array,BitInfoDic = GenerateFP(smiles,names)

# Generate X,y set
# FP
FP_array = FP_array.astype(int)
FP_df = pd.DataFrame(data=FP_array,columns= ['FP_'+str(i) for i in range(1,1025)],index=names)

# Normalize descriptor
from sklearn import preprocessing
des_df=f.iloc[:,4:-1]
des_df.index = names
# min_max_scaler = preprocessing.MinMaxScaler()
# des_np = min_max_scaler.fit_transform(des_df)
# des_df = pd.DataFrame(des_np,index = names, columns = des_df.columns)
# print(des_df)

X = pd.concat([des_df,FP_df],axis=1)
y = f.iloc[:,-1]
X.to_csv(r'data/NoNormlization/X_%s.csv'%datatype)
y.to_csv(r'data/NoNormlization/y_%s.csv'%datatype,index=None)

fout = open(r'data/NoNormlization/%s_chemical_FPBitInfoDic.pickle'%datatype,'wb')
pickle.dump(BitInfoDic,fout)
fout.close()